In [1]:
#Dependencies and Setup
import pandas as pd
import numpy as np
import os
import warnings
from sqlalchemy import create_engine
warnings.filterwarnings('ignore')

In [2]:
#Load CSV
IRS_data = os.path.join('resources', 'IRS_Data_2015_clean.csv')
IRS_data = pd.read_csv(IRS_data)

police_data = os.path.join('resources', 'police_data.csv')
police_data = pd.read_csv(police_data)

In [3]:
#Police Data clean up

In [4]:
police_data["ReportedDate"] = pd.to_datetime(police_data["ReportedDate"]).dt.date

In [5]:
police_data["ReportedDate"] = pd.to_datetime(police_data["ReportedDate"])

In [6]:
police_data.drop(["FID", 
           "ControlNbr",
           "CCN", 
           "BeginDate",
           "Time", 
           "UCRCode", 
           "EnteredDate", 
           "GBSID", 
           "X",
           "Y",
           "Lat",
           "Long",
           "LastChanged", 
           "LastUpdateDate"], axis=1, inplace=True)

police_data.drop("Unnamed: 0", axis=1, inplace=True)

police_data.rename(columns={"ReportedDate": "incidentdate", 
                     "PublicAddress":"publicaddress",
                     "Precinct":"precinct",
                     "Offense":"offense",
                     "Description":"description",
                     "Neighborhood":"neighborhood",
                     "Zip Code":"zipcode"}, inplace=True)

In [7]:
police_data.to_csv("cleaned_police_data.csv", index=False)

In [8]:
#IRS data clean up

In [9]:
IRS_data = IRS_data.dropna()

In [10]:
IRS_data_fliterd = IRS_data.drop_duplicates('zipcode')

In [11]:
IRS_data_fliterd.replace(',','', regex=True, inplace=True)

In [12]:
#Postgres business

In [13]:
username = input('What is your PGAmin user name? The default user name is postgres.')

What is your PGAmin user name? The default user name is postgres.


In [14]:
if username == "":
     username = "postgres"

In [15]:
password = input('what is your PGAdmin password?')

what is your PGAdmin password?1234


In [16]:
#Input PGAdmin user name and password below
rds_connection_string = f"{username}:{password}@localhost:5432/ETL"
#rds_connection_string ="postgres:1234@localhost:5432/ETL"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [ ]:
engine.table_names()

In [ ]:
police_data.to_sql(name='offenses', con=engine, if_exists='append', index=False)

In [ ]:
IRS_data_fliterd.to_sql(name='irsdata', con=engine, if_exists='append', index=False)

In [ ]:
pd.read_sql_query('select * from offenses', con=engine).head()

In [ ]:
pd.read_sql_query('select * from irsdata', con=engine).head()